In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
url = 'https://www.amazon.com/s?k=jacket+for+men&ref=nb_sb_noss'

In [3]:
r = requests.get('http://localhost:8050/render.html', {'url': url, 'wait': 2})
#print(r.text)

ConnectionError: HTTPConnectionPool(host='localhost', port=8050): Max retries exceeded with url: /render.html?url=https%3A%2F%2Fwww.amazon.com%2Fs%3Fk%3Djacket%2Bfor%2Bmen%26ref%3Dnb_sb_noss&wait=2 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001ED495BAEE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
#soup

In [93]:
product_links = pd.DataFrame(columns=['product', 'link'])
i = 0

url = 'https://www.amazon.com/s?k=jacket+for+men&ref=nb_sb_noss'

for page in ranlige(1, 1000):

    print("scraping page - ", page)
    
    r = requests.get('http://localhost:8050/render.html', {'url': url, 'wait': 3})
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # stop if next page button is disabled
    if soup.find('li', {'class': 'a-disabled a-last'}):
        break
    else:
        # go to next page
        # find element of next page
        next_links = soup.find_all('li', {'class':'a-last'})
        for link in next_links:
            url = 'https://amazon.com' + str(link.find('a')['href'])
    
    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    for product in products:
        item = product.find('a', {'class':'a-link-normal s-underline-text s-underline-link-text a-text-normal'})
        if item == None:
            item = product.find('a', {'class':'a-link-normal a-text-normal'})
        title = item.text.strip()
        link = item['href']
        product_links.loc[i] = [title, link]
        i = i+1
    if not page == 1:
        soup = soup.find_all('li', {'class':'a-normal'})

print('done')
product_links.to_csv('scraped_product_links.csv', index=False)

scraping page -  1
scraping page -  2
scraping page -  3
scraping page -  4
scraping page -  5
scraping page -  6
scraping page -  7
done


In [37]:
products = pd.read_csv('scraped_product_links.csv')

products['features'] = ''
products['description'] = ''
products['review_pages'] = ''

for i in range(len(products)):
    
    print("", round((100 * i) / len(products)), "% product pages scraped")
    
    url_base = products['link'].iloc[i]
    url = 'https://amazon.com' + url_base

    r = requests.get('http://localhost:8000/render.html', {'url': url, 'wait': 7})

    soup = BeautifulSoup(r.text, 'html.parser')

    product_features = ''
    product_description = ''
    
    review_item = soup.find({'a'}, {'data-hook':"see-all-reviews-link-foot"})
    
    if review_item is None:
        # if there are not enough reviews we will skip over
        products = products.drop(axis = 0, index = i)
        i = i-1
        print('Product removed from list: not enough reviews')
        continue
    review_page_link = soup.find({'a'}, {'data-hook':"see-all-reviews-link-foot"})['href']

    feature_list_conatiner = soup.find({'div'}, {'id':"feature-bullets"})
    feature_list = feature_list_conatiner.find_all({'span'}, {'class':"a-list-item"})
    for feature in feature_list:
        product_features = product_features + " " + feature.text.strip()

    product_description_container = soup.find('div', 
                                              {'cel_widget_id':
                                               "dpx-aplus-3p-product-description_csm_instrumentation_wrapper"})    
    
    if product_description_container == None:
        product_description_container = soup.find('div', {'id':'productDescription'})

    if product_description_container == None:
        products['features'][i] = product_features
        products['description'][i] = product_description
        products['review_pages'][i] = review_page_link
        continue
    
    lists = product_description_container.find_all({'span'}, {'class':"a-list-item"})
    
    for item in lists:
        product_description = product_description + " " + item.text.strip()

    paras = product_description_container.find_all('p')
    for para in paras:
        if '<img alt=' in para.text:
            continue
        product_description = product_description + " " + para.text.strip()
    
    products['features'][i] = product_features
    products['description'][i] = product_description
    products['review_pages'][i] = review_page_link

 0 % product pages scraped
Product removed from list: not enough reviews
 0 % product pages scraped
Product removed from list: not enough reviews
 1 % product pages scraped


KeyboardInterrupt: 

In [ ]:
products.to_csv('products_complete.csv', index = False)

In [ ]:
def clean_title(item):
    if item == None:
        return ""
    else:
        return item.text.strip().replace('\n', '')
    
def clean_rating(item):
    if item == None:
        return 0.0
    else:
        return float(item.text[:3])

def clean_body(item):
    if item == None:
        return ""
    else:
        return item.text.strip()
df = pd.DataFrame(columns = ['product', 'review_title', 'review_rating', 'review_text'])

i = 0

products = pd.read_csv('products_complete.csv')

for prd_idx in range(len(products)):
    print('scraping reviews of product - ', prd_idx + 1)
    url_base = products['review_pages'][prd_idx]
    
    if url_base = 
    
    for page in range(1,100):
        url = 'https://amazon.com' + url_base + str(page)
        r = requests.get('http://localhost:8051/render.html', {'url': url, 'wait': 3})
        soup = BeautifulSoup(r.text, 'html.parser')

        #check if we have reached last page
        if soup.find('li', {'class': 'a-disabled a-last'}):
            break

        reviews = soup.find_all('div', {'data-hook': 'review'})

        for items in reviews:
            product = soup.title.text.replace('Amazon.com: Customer reviews:', '').strip()
            reviews_title= clean_title(items.find('a', {'data-hook': 'review-title'}))
            review_rating= clean_rating(items.find('i', {'data-hook': 'review-star-rating'}))
            reviews_text= clean_body(items.find('span', {'data-hook': 'review-body'}))
            df.loc[i] = [product, reviews_title, review_rating, reviews_text]
            i = i + 1

print("Done scraping")        
df.to_csv('scraped_data.csv')